# Customer.IO Data Pipelines API - Device Management

## Purpose

This notebook demonstrates comprehensive device registration and management with Customer.IO's Data Pipelines API.
It covers mobile device registration, push notification setup, device attribute management, and cross-platform device tracking with proper validation and error handling.

## Prerequisites

- Complete setup from `00_setup_and_configuration.ipynb`
- Complete authentication setup from `01_authentication_and_utilities.ipynb`
- Customer.IO API key configured in Databricks secrets
- Understanding of mobile push notification concepts

## Key Concepts

- **Device Registration**: Linking devices to users for push notifications
- **Push Tokens**: Device-specific identifiers for notification delivery
- **Platform Types**: iOS, Android, and web push notification differences
- **Device Attributes**: Custom properties for targeting and segmentation
- **Token Management**: Registration, updates, and cleanup
- **Cross-Platform Tracking**: Users with multiple devices

## Device Types Covered

1. **iOS Devices**: APNS (Apple Push Notification Service) integration
2. **Android Devices**: FCM (Firebase Cloud Messaging) integration
3. **Web Browsers**: Web Push API for browser notifications
4. **Desktop Applications**: Cross-platform desktop push notifications
5. **Smart Devices**: IoT and embedded device tracking

## Setup and Imports

In [ ]:
# Standard library imports
import sys
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any, Union, Set
import json
import uuid
from enum import Enum
import re

print("SUCCESS: Standard libraries imported")

In [ ]:
# Add utils directory to Python path
sys.path.append('/Workspace/Repos/customer_io_notebooks/utils')
print("SUCCESS: Utils directory added to Python path")

In [ ]:
# Import Customer.IO API utilities
from utils.api_client import CustomerIOClient
from utils.validators import (
    DeviceRequest,
    validate_request_size,
    create_context
)

print("SUCCESS: Customer.IO API utilities imported")

In [ ]:
# Import transformation utilities
from utils.transformers import (
    BatchTransformer,
    ContextTransformer
)

print("SUCCESS: Transformation utilities imported")

In [ ]:
# Import error handling utilities
from utils.error_handlers import (
    CustomerIOError,
    RateLimitError,
    ValidationError,
    NetworkError,
    retry_on_error,
    ErrorContext
)

print("SUCCESS: Error handling utilities imported")

In [ ]:
# Import Databricks and Spark utilities
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable

print("SUCCESS: Databricks and Spark utilities imported")

In [ ]:
# Import validation and logging
import structlog
from pydantic import ValidationError as PydanticValidationError, BaseModel, Field, validator

# Initialize logger
logger = structlog.get_logger("device_management")

print("SUCCESS: Validation and logging initialized")

## Configuration and Client Setup

In [ ]:
# Load configuration from setup notebook (secure approach)
try:
    CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
    DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
    CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
    ENVIRONMENT = dbutils.widgets.get("environment") or "test"
    
    print(f"Configuration loaded from setup notebook:")
    print(f"  Region: {CUSTOMERIO_REGION}")
    print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
    print(f"  Environment: {ENVIRONMENT}")
    
except Exception as e:
    print(f"WARNING: Could not load configuration from setup notebook: {str(e)}")
    print("INFO: Using fallback configuration")
    CUSTOMERIO_REGION = "us"
    DATABASE_NAME = "customerio_demo"
    CATALOG_NAME = "main"
    ENVIRONMENT = "test"

In [ ]:
# Get Customer.IO API key from secure storage
CUSTOMERIO_API_KEY = dbutils.secrets.get("customerio", "api_key")
print("SUCCESS: Customer.IO API key retrieved from secure storage")

In [ ]:
# Configure Spark to use the specified database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")
print("SUCCESS: Database configured")

In [ ]:
# Initialize the Customer.IO client
try:
    client = CustomerIOClient(
        api_key=CUSTOMERIO_API_KEY,
        region=CUSTOMERIO_REGION,
        timeout=30,
        max_retries=3,
        retry_backoff_factor=2.0,
        enable_logging=True,
        spark_session=spark
    )
    print("SUCCESS: Customer.IO client initialized for device management")
    
except Exception as e:
    print(f"ERROR: Failed to initialize Customer.IO client: {str(e)}")
    raise

## Test-Driven Development: Device Validation Functions

In [ ]:
# Test function: Validate device registration structure
def test_device_registration_validation():
    """Test that device registrations have required fields and pass validation."""
    
    # Test valid iOS device
    valid_ios_device = {
        "device": {
            "token": "abc123def456ghi789",
            "type": "ios",
            "device_id": "iPhone14,2",
            "os_version": "17.2",
            "app_version": "2.1.0"
        }
    }
    
    try:
        device_request = DeviceRequest(**valid_ios_device)
        assert device_request.device["type"] == "ios"
        assert device_request.device["token"] == "abc123def456ghi789"
        print("SUCCESS: iOS device validation test passed")
        return True
    except Exception as e:
        print(f"ERROR: iOS device validation test failed: {str(e)}")
        return False

# Run the test
test_device_registration_validation()

In [ ]:
# Test function: Validate push token formats
def test_push_token_validation():
    """Test that push tokens are validated for different platforms."""
    
    # Test iOS token format (64 hex characters)
    ios_token = "a1b2c3d4e5f6789012345678901234567890123456789012345678901234abcd"
    if len(ios_token) != 64 or not all(c in '0123456789abcdefABCDEF' for c in ios_token):
        print("ERROR: iOS token format validation failed")
        return False
    
    # Test Android token format (FCM token - variable length)
    android_token = "fGcm_token_123:APA91bH...example_token"
    if len(android_token) < 10:  # Minimum reasonable length
        print("ERROR: Android token too short")
        return False
    
    # Test web push token format
    web_token = "BP91bH_example_web_push_token_with_base64_encoding"
    if len(web_token) < 10:
        print("ERROR: Web token too short")
        return False
    
    print("SUCCESS: Push token validation tests passed")
    return True

# Run the test
test_push_token_validation()

In [ ]:
# Test function: Validate device attribute structure
def test_device_attributes_validation():
    """Test that device attributes are properly structured and typed."""
    
    # Test device attributes
    device_attributes = {
        "device_model": "iPhone 15 Pro",
        "os_version": "17.2",
        "app_version": "2.1.0",
        "push_enabled": True,
        "timezone": "America/New_York",
        "locale": "en-US",
        "battery_level": 0.85,
        "network_type": "wifi"
    }
    
    # Validate required attributes
    required = ["device_model", "os_version", "app_version"]
    for attr in required:
        if attr not in device_attributes:
            print(f"ERROR: Missing required attribute: {attr}")
            return False
    
    # Validate data types
    if not isinstance(device_attributes["push_enabled"], bool):
        print("ERROR: push_enabled must be boolean")
        return False
    
    if not (0.0 <= device_attributes["battery_level"] <= 1.0):
        print("ERROR: battery_level must be between 0.0 and 1.0")
        return False
    
    print("SUCCESS: Device attributes validation test passed")
    return True

# Run the test
test_device_attributes_validation()

## Device Types and Platform Support

In [ ]:
# Define device types and platform enumerations
class DeviceType(str, Enum):
    """Enumeration for device types."""
    IOS = "ios"
    ANDROID = "android"
    WEB = "web"
    DESKTOP = "desktop"
    IOT = "iot"

class PushProvider(str, Enum):
    """Enumeration for push notification providers."""
    APNS = "apns"  # Apple Push Notification Service
    FCM = "fcm"    # Firebase Cloud Messaging
    WEB_PUSH = "web_push"  # Web Push API
    DESKTOP = "desktop"    # Desktop notifications
    CUSTOM = "custom"      # Custom push provider

class DeviceStatus(str, Enum):
    """Enumeration for device status."""
    ACTIVE = "active"
    INACTIVE = "inactive"
    UNINSTALLED = "uninstalled"
    OPT_OUT = "opt_out"
    INVALID_TOKEN = "invalid_token"

print("SUCCESS: Device types and enumerations defined")

## Type-Safe Device Models

In [ ]:
# Define type-safe device models
class DeviceInfo(BaseModel):
    """Type-safe device information model."""
    token: str = Field(..., description="Push notification token")
    type: DeviceType = Field(..., description="Device type")
    device_id: Optional[str] = Field(None, description="Unique device identifier")
    device_model: Optional[str] = Field(None, description="Device model name")
    os_version: Optional[str] = Field(None, description="Operating system version")
    app_version: Optional[str] = Field(None, description="Application version")
    push_provider: Optional[PushProvider] = Field(None, description="Push provider")
    
    @validator('token')
    def validate_token(cls, v: str) -> str:
        """Validate push token format."""
        if not v or len(v.strip()) == 0:
            raise ValueError("Push token cannot be empty")
        if len(v) < 10:  # Minimum reasonable token length
            raise ValueError("Push token appears to be too short")
        return v.strip()
    
    class Config:
        """Pydantic model configuration."""
        use_enum_values = True
        validate_assignment = True

print("SUCCESS: DeviceInfo model defined with validation")

In [ ]:
# Define device attributes model
class DeviceAttributes(BaseModel):
    """Type-safe device attributes model."""
    push_enabled: bool = Field(default=True, description="Push notifications enabled")
    timezone: Optional[str] = Field(None, description="Device timezone")
    locale: Optional[str] = Field(None, description="Device locale")
    battery_level: Optional[float] = Field(None, ge=0.0, le=1.0, description="Battery level (0.0-1.0)")
    network_type: Optional[str] = Field(None, description="Network connection type")
    carrier: Optional[str] = Field(None, description="Mobile carrier")
    screen_width: Optional[int] = Field(None, ge=0, description="Screen width in pixels")
    screen_height: Optional[int] = Field(None, ge=0, description="Screen height in pixels")
    last_seen: Optional[datetime] = Field(None, description="Last activity timestamp")
    
    @validator('timezone')
    def validate_timezone(cls, v: Optional[str]) -> Optional[str]:
        """Validate timezone format."""
        if v and '/' not in v:
            # Simple validation - proper timezone should have region/city format
            raise ValueError("Timezone should be in region/city format (e.g., America/New_York)")
        return v
    
    @validator('locale')
    def validate_locale(cls, v: Optional[str]) -> Optional[str]:
        """Validate locale format."""
        if v and not re.match(r'^[a-z]{2}(-[A-Z]{2})?$', v):
            raise ValueError("Locale should be in format 'en' or 'en-US'")
        return v

print("SUCCESS: DeviceAttributes model defined")

In [ ]:
# Define device registration model
class DeviceRegistration(BaseModel):
    """Type-safe device registration model."""
    user_id: str = Field(..., description="User identifier")
    device_info: DeviceInfo = Field(..., description="Device information")
    attributes: DeviceAttributes = Field(default_factory=DeviceAttributes, description="Device attributes")
    status: DeviceStatus = Field(default=DeviceStatus.ACTIVE, description="Device status")
    registered_at: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))
    
    @validator('user_id')
    def validate_user_id(cls, v: str) -> str:
        """Validate user ID format."""
        if not v or len(v.strip()) == 0:
            raise ValueError("User ID cannot be empty")
        return v.strip()
    
    class Config:
        """Pydantic model configuration."""
        use_enum_values = True
        validate_assignment = True

print("SUCCESS: DeviceRegistration model defined")

## Device Registration Implementation

In [ ]:
# Implementation: Register device with Customer.IO
def register_device(
    device_registration: DeviceRegistration
) -> Dict[str, Any]:
    """Register a device with Customer.IO for push notifications."""
    
    # Create device request
    device_data = {
        "device": {
            "token": device_registration.device_info.token,
            "type": device_registration.device_info.type,
            **device_registration.device_info.dict(exclude={'token', 'type'}, exclude_none=True)
        }
    }
    
    # Validate with DeviceRequest model
    device_request = DeviceRequest(**device_data)
    
    return device_request.dict()

# Test device registration
ios_device_info = DeviceInfo(
    token="a1b2c3d4e5f6789012345678901234567890123456789012345678901234abcd",
    type=DeviceType.IOS,
    device_id="iPhone14,2",
    device_model="iPhone 15 Pro",
    os_version="17.2",
    app_version="2.1.0",
    push_provider=PushProvider.APNS
)

ios_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/New_York",
    locale="en-US",
    battery_level=0.85,
    network_type="wifi",
    screen_width=1179,
    screen_height=2556
)

ios_registration = DeviceRegistration(
    user_id="user_mobile_001",
    device_info=ios_device_info,
    attributes=ios_attributes,
    status=DeviceStatus.ACTIVE
)

device_data = register_device(ios_registration)
print("iOS device registration:")
print(json.dumps(device_data, indent=2, default=str))

In [ ]:
# Implementation: Send device registration to Customer.IO
def send_device_registration(
    user_id: str,
    device_data: Dict[str, Any],
    test_mode: bool = True
) -> Dict[str, Any]:
    """Send device registration to Customer.IO with error handling."""
    
    try:
        if test_mode:
            print("INFO: Running in test mode - device not actually registered")
            return {"status": "test_success", "message": "Device registration validated"}
        
        # Send device registration
        response = client.add_device(user_id=user_id, **device_data)
        
        logger.info(
            "Device registered successfully",
            user_id=user_id,
            device_type=device_data["device"]["type"]
        )
        
        return response
        
    except CustomerIOError as e:
        logger.error("Failed to register device", error=str(e))
        raise

# Test sending device registration
result = send_device_registration(
    user_id=ios_registration.user_id,
    device_data=device_data,
    test_mode=(ENVIRONMENT == "test")
)
print(f"Device registration result: {result}")

## Multi-Platform Device Support

In [ ]:
# Implementation: Create Android device registration
android_device_info = DeviceInfo(
    token="fGcm_token_123:APA91bH_example_android_fcm_token_with_proper_format",
    type=DeviceType.ANDROID,
    device_id="android_device_001",
    device_model="Samsung Galaxy S24",
    os_version="14.0",
    app_version="2.1.0",
    push_provider=PushProvider.FCM
)

android_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/Los_Angeles",
    locale="en-US",
    battery_level=0.92,
    network_type="5g",
    carrier="Verizon",
    screen_width=1440,
    screen_height=3120
)

android_registration = DeviceRegistration(
    user_id="user_mobile_002",
    device_info=android_device_info,
    attributes=android_attributes,
    status=DeviceStatus.ACTIVE
)

android_device_data = register_device(android_registration)
print("Android device registration:")
print(json.dumps(android_device_data, indent=2, default=str))

In [ ]:
# Implementation: Create web browser device registration
web_device_info = DeviceInfo(
    token="BP91bH_example_web_push_token_with_base64_encoding_for_browser_notifications",
    type=DeviceType.WEB,
    device_id="web_browser_001",
    device_model="Chrome Browser",
    os_version="macOS 14.2",
    app_version="121.0.6167.85",
    push_provider=PushProvider.WEB_PUSH
)

web_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/Chicago",
    locale="en-US",
    network_type="wifi",
    screen_width=1920,
    screen_height=1080
)

web_registration = DeviceRegistration(
    user_id="user_web_001",
    device_info=web_device_info,
    attributes=web_attributes,
    status=DeviceStatus.ACTIVE
)

web_device_data = register_device(web_registration)
print("Web device registration:")
print(json.dumps(web_device_data, indent=2, default=str))

In [ ]:
# Implementation: Create desktop application device registration
desktop_device_info = DeviceInfo(
    token="desktop_push_token_example_for_native_desktop_application_notifications",
    type=DeviceType.DESKTOP,
    device_id="desktop_app_001",
    device_model="MacBook Pro",
    os_version="macOS 14.2",
    app_version="3.0.1",
    push_provider=PushProvider.DESKTOP
)

desktop_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/New_York",
    locale="en-US",
    network_type="ethernet",
    screen_width=2560,
    screen_height=1600
)

desktop_registration = DeviceRegistration(
    user_id="user_desktop_001",
    device_info=desktop_device_info,
    attributes=desktop_attributes,
    status=DeviceStatus.ACTIVE
)

desktop_device_data = register_device(desktop_registration)
print("Desktop device registration:")
print(json.dumps(desktop_device_data, indent=2, default=str))

## Device Lifecycle Management

In [ ]:
# Implementation: Update device status
def update_device_status(
    user_id: str,
    device_token: str,
    new_status: DeviceStatus,
    reason: Optional[str] = None
) -> Dict[str, Any]:
    """Update device status with tracking event."""
    
    event_data = {
        "userId": user_id,
        "event": "Device Status Updated",
        "properties": {
            "device_token": device_token[:20] + "...",  # Truncate for privacy
            "new_status": new_status,
            "updated_at": datetime.now(timezone.utc).isoformat()
        },
        "timestamp": datetime.now(timezone.utc)
    }
    
    if reason:
        event_data["properties"]["reason"] = reason
    
    return event_data

# Test device status update
status_update = update_device_status(
    user_id="user_mobile_001",
    device_token=ios_device_info.token,
    new_status=DeviceStatus.INACTIVE,
    reason="User disabled push notifications"
)

print("Device status update event:")
print(json.dumps(status_update, indent=2, default=str))

In [ ]:
# Implementation: Handle device token refresh
def refresh_device_token(
    user_id: str,
    old_token: str,
    new_token: str,
    device_type: DeviceType
) -> Dict[str, Any]:
    """Handle device token refresh with proper tracking."""
    
    event_data = {
        "userId": user_id,
        "event": "Device Token Refreshed",
        "properties": {
            "old_token": old_token[:20] + "...",  # Truncate for privacy
            "new_token": new_token[:20] + "...",  # Truncate for privacy
            "device_type": device_type,
            "refreshed_at": datetime.now(timezone.utc).isoformat()
        },
        "timestamp": datetime.now(timezone.utc)
    }
    
    return event_data

# Test token refresh
new_ios_token = "z9y8x7w6v5u4t3s2r1q0p9o8n7m6l5k4j3i2h1g0f9e8d7c6b5a4321098765432"
token_refresh = refresh_device_token(
    user_id="user_mobile_001",
    old_token=ios_device_info.token,
    new_token=new_ios_token,
    device_type=DeviceType.IOS
)

print("Device token refresh event:")
print(json.dumps(token_refresh, indent=2, default=str))

In [ ]:
# Implementation: Remove device registration
def remove_device(
    user_id: str,
    device_token: str,
    device_type: DeviceType,
    reason: str = "user_request"
) -> Dict[str, Any]:
    """Remove device registration with tracking."""
    
    event_data = {
        "userId": user_id,
        "event": "Device Removed",
        "properties": {
            "device_token": device_token[:20] + "...",  # Truncate for privacy
            "device_type": device_type,
            "reason": reason,
            "removed_at": datetime.now(timezone.utc).isoformat()
        },
        "timestamp": datetime.now(timezone.utc)
    }
    
    return event_data

# Test device removal
device_removal = remove_device(
    user_id="user_mobile_002",
    device_token=android_device_info.token,
    device_type=DeviceType.ANDROID,
    reason="app_uninstalled"
)

print("Device removal event:")
print(json.dumps(device_removal, indent=2, default=str))

## Batch Device Operations

In [ ]:
# Implementation: Batch register multiple devices
def batch_register_devices(
    registrations: List[DeviceRegistration]
) -> List[Dict[str, Any]]:
    """Register multiple devices in batch."""
    
    events = []
    
    for registration in registrations:
        # Create device registration event
        event_data = {
            "userId": registration.user_id,
            "event": "Device Registered",
            "properties": {
                "device_type": registration.device_info.type,
                "device_model": registration.device_info.device_model,
                "os_version": registration.device_info.os_version,
                "app_version": registration.device_info.app_version,
                "push_provider": registration.device_info.push_provider,
                "push_enabled": registration.attributes.push_enabled,
                "timezone": registration.attributes.timezone,
                "locale": registration.attributes.locale,
                "registered_at": registration.registered_at.isoformat()
            },
            "timestamp": datetime.now(timezone.utc)
        }
        
        events.append(event_data)
    
    return events

# Create sample device registrations for batch
batch_registrations = [
    ios_registration,
    android_registration,
    web_registration,
    desktop_registration
]

batch_events = batch_register_devices(batch_registrations)
print(f"Created {len(batch_events)} device registration events for batch:")
for event in batch_events:
    device_type = event["properties"]["device_type"]
    user_id = event["userId"]
    print(f"  - {user_id}: {device_type} device")

In [ ]:
# Implementation: Send batch device registrations
@retry_on_error(max_retries=3, backoff_factor=2.0)
def send_device_batch(
    events: List[Dict[str, Any]],
    test_mode: bool = True
) -> List[Dict[str, Any]]:
    """Send device events in optimized batches."""
    
    try:
        # Create batch requests
        batch_requests = [{"type": "track", **event} for event in events]
        
        # Optimize batch sizes
        optimized_batches = BatchTransformer.optimize_batch_sizes(
            requests=batch_requests,
            max_size_bytes=500 * 1024  # 500KB limit
        )
        
        print(f"Optimized {len(events)} device events into {len(optimized_batches)} batch(es)")
        
        results = []
        
        # Process each batch
        for i, batch in enumerate(optimized_batches):
            try:
                if test_mode:
                    print(f"  Batch {i+1}: {len(batch)} events (test mode)")
                    results.append({
                        "batch_id": i,
                        "status": "test_success",
                        "count": len(batch)
                    })
                else:
                    response = client.batch(batch)
                    results.append({
                        "batch_id": i,
                        "status": "success",
                        "count": len(batch),
                        "response": response
                    })
                    
            except Exception as e:
                results.append({
                    "batch_id": i,
                    "status": "failed",
                    "count": len(batch),
                    "error": str(e)
                })
                logger.error(f"Device batch {i} failed", error=str(e))
        
        return results
        
    except Exception as e:
        logger.error("Device batch processing failed", error=str(e))
        raise

# Send device batch
batch_results = send_device_batch(
    events=batch_events,
    test_mode=(ENVIRONMENT == "test")
)

print("\nDevice batch submission results:")
for result in batch_results:
    print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} events)")

## Cross-Platform Device Tracking

In [ ]:
# Implementation: Track user across multiple devices
def create_cross_platform_user(
    user_id: str,
    devices: List[DeviceRegistration]
) -> Dict[str, Any]:
    """Create tracking for user with multiple devices."""
    
    # Analyze device portfolio
    device_types = [device.device_info.type for device in devices]
    platforms = list(set(device_types))
    
    # Create cross-platform event
    event_data = {
        "userId": user_id,
        "event": "Multi-Device User Profile",
        "properties": {
            "total_devices": len(devices),
            "platforms": platforms,
            "device_breakdown": {
                device_type: device_types.count(device_type)
                for device_type in set(device_types)
            },
            "primary_platform": max(set(device_types), key=device_types.count),
            "cross_platform": len(platforms) > 1,
            "created_at": datetime.now(timezone.utc).isoformat()
        },
        "timestamp": datetime.now(timezone.utc)
    }
    
    return event_data

# Create cross-platform user
multi_device_user_devices = [
    ios_registration,
    android_registration,
    web_registration
]

# Update all devices to same user for demonstration
for device in multi_device_user_devices:
    device.user_id = "user_cross_platform_001"

cross_platform_event = create_cross_platform_user(
    user_id="user_cross_platform_001",
    devices=multi_device_user_devices
)

print("Cross-platform user profile:")
print(json.dumps(cross_platform_event, indent=2, default=str))

In [ ]:
# Implementation: Device preference analysis
def analyze_device_preferences(
    devices: List[DeviceRegistration]
) -> Dict[str, Any]:
    """Analyze device usage patterns and preferences."""
    
    # Group devices by type
    by_type = {}
    for device in devices:
        device_type = device.device_info.type
        if device_type not in by_type:
            by_type[device_type] = []
        by_type[device_type].append(device)
    
    # Analyze preferences
    analysis = {
        "total_devices": len(devices),
        "device_types": list(by_type.keys()),
        "type_distribution": {k: len(v) for k, v in by_type.items()},
        "push_enabled_count": sum(1 for d in devices if d.attributes.push_enabled),
        "active_devices": sum(1 for d in devices if d.status == DeviceStatus.ACTIVE),
        "unique_timezones": len(set(
            d.attributes.timezone for d in devices 
            if d.attributes.timezone
        )),
        "unique_locales": len(set(
            d.attributes.locale for d in devices 
            if d.attributes.locale
        ))
    }
    
    # Find most recent device
    if devices:
        most_recent = max(devices, key=lambda d: d.registered_at)
        analysis["most_recent_device"] = {
            "type": most_recent.device_info.type,
            "model": most_recent.device_info.device_model,
            "registered_at": most_recent.registered_at.isoformat()
        }
    
    return analysis

# Analyze device preferences
all_devices = [ios_registration, android_registration, web_registration, desktop_registration]
preferences = analyze_device_preferences(all_devices)

print("Device preference analysis:")
print(json.dumps(preferences, indent=2, default=str))

## Device Data from Spark Integration

In [ ]:
# Load device data from Delta table
print("=== Device Data Integration ===")

# Create sample device data table if it doesn't exist
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.device_registrations (
    user_id STRING,
    device_token STRING,
    device_type STRING,
    device_model STRING,
    os_version STRING,
    app_version STRING,
    push_enabled BOOLEAN,
    timezone STRING,
    locale STRING,
    registered_at TIMESTAMP
) USING DELTA
""")

# Insert sample device data
spark.sql(f"""
INSERT INTO {CATALOG_NAME}.{DATABASE_NAME}.device_registrations
SELECT * FROM VALUES
    ('user_spark_001', 'ios_token_001', 'ios', 'iPhone 15', '17.2', '2.1.0', true, 'America/New_York', 'en-US', current_timestamp()),
    ('user_spark_001', 'android_token_001', 'android', 'Pixel 8', '14.0', '2.1.0', true, 'America/New_York', 'en-US', current_timestamp()),
    ('user_spark_002', 'web_token_001', 'web', 'Chrome', '121.0', '2.1.0', true, 'America/Los_Angeles', 'en-US', current_timestamp()),
    ('user_spark_003', 'ios_token_002', 'ios', 'iPhone 14', '17.1', '2.0.8', false, 'Europe/London', 'en-GB', current_timestamp())
WHERE NOT EXISTS (
    SELECT 1 FROM {CATALOG_NAME}.{DATABASE_NAME}.device_registrations 
    WHERE device_token = 'ios_token_001'
)
""")

# Load device registrations
devices_df = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.device_registrations")
print("Sample device registrations from Spark:")
devices_df.show(truncate=False)

In [ ]:
# Transform Spark device data to Customer.IO format
def transform_spark_devices_to_events(df):
    """Transform device data from Spark to Customer.IO events."""
    
    # Collect data (in production, process in batches)
    devices = df.collect()
    
    events = []
    for device in devices:
        event_data = {
            "userId": device['user_id'],
            "event": "Device Registered from Data",
            "properties": {
                "device_type": device['device_type'],
                "device_model": device['device_model'],
                "os_version": device['os_version'],
                "app_version": device['app_version'],
                "push_enabled": device['push_enabled'],
                "timezone": device['timezone'],
                "locale": device['locale'],
                "data_source": "spark_etl",
                "imported_at": datetime.now(timezone.utc).isoformat()
            },
            "timestamp": device['registered_at']
        }
        
        events.append(event_data)
    
    return events

# Transform device data
spark_device_events = transform_spark_devices_to_events(devices_df)
print(f"Transformed {len(spark_device_events)} device records to events")

# Show sample
if spark_device_events:
    print("\nSample transformed device event:")
    print(json.dumps(spark_device_events[0], indent=2, default=str))

In [ ]:
# Process Spark device events in batch
if spark_device_events:
    spark_batch_results = send_device_batch(
        events=spark_device_events,
        test_mode=(ENVIRONMENT == "test")
    )
    
    print("\nSpark device events batch results:")
    for result in spark_batch_results:
        print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} events)")
else:
    print("No device events to process from Spark data")

## Device Analytics and Reporting

In [ ]:
# Implementation: Device analytics
def calculate_device_metrics(
    registrations: List[DeviceRegistration]
) -> Dict[str, Any]:
    """Calculate comprehensive device metrics."""
    
    if not registrations:
        return {"total_devices": 0}
    
    # Basic counts
    total_devices = len(registrations)
    active_devices = sum(1 for d in registrations if d.status == DeviceStatus.ACTIVE)
    push_enabled = sum(1 for d in registrations if d.attributes.push_enabled)
    
    # Platform distribution
    platform_counts = {}
    for device in registrations:
        platform = device.device_info.type
        platform_counts[platform] = platform_counts.get(platform, 0) + 1
    
    # Push provider distribution
    provider_counts = {}
    for device in registrations:
        provider = device.device_info.push_provider or "unknown"
        provider_counts[provider] = provider_counts.get(provider, 0) + 1
    
    # Timezone and locale analysis
    timezones = [d.attributes.timezone for d in registrations if d.attributes.timezone]
    locales = [d.attributes.locale for d in registrations if d.attributes.locale]
    
    # Registration timeline
    if registrations:
        newest = max(registrations, key=lambda d: d.registered_at)
        oldest = min(registrations, key=lambda d: d.registered_at)
        
        registration_span = (newest.registered_at - oldest.registered_at).days
    else:
        registration_span = 0
    
    metrics = {
        "total_devices": total_devices,
        "active_devices": active_devices,
        "inactive_devices": total_devices - active_devices,
        "push_enabled_devices": push_enabled,
        "push_opt_out_devices": total_devices - push_enabled,
        "platform_distribution": platform_counts,
        "push_provider_distribution": provider_counts,
        "unique_timezones": len(set(timezones)),
        "unique_locales": len(set(locales)),
        "registration_span_days": registration_span,
        "avg_devices_per_day": (
            total_devices / max(registration_span, 1) if registration_span > 0 else total_devices
        ),
        "push_adoption_rate": push_enabled / total_devices if total_devices > 0 else 0,
        "active_device_rate": active_devices / total_devices if total_devices > 0 else 0
    }
    
    return metrics

# Calculate metrics for all devices
all_device_registrations = [
    ios_registration,
    android_registration,
    web_registration,
    desktop_registration
]

device_metrics = calculate_device_metrics(all_device_registrations)

print("=== Device Analytics ===")
print(json.dumps(device_metrics, indent=2, default=str))

## Performance Monitoring and Health Checks

In [ ]:
# Implementation: Device health monitoring
def monitor_device_health(
    registrations: List[DeviceRegistration]
) -> Dict[str, Any]:
    """Monitor device health and identify issues."""
    
    health_report = {
        "total_devices": len(registrations),
        "healthy_devices": 0,
        "issues_found": [],
        "recommendations": []
    }
    
    for device in registrations:
        device_issues = []
        
        # Check device status
        if device.status != DeviceStatus.ACTIVE:
            device_issues.append(f"Device status: {device.status}")
        
        # Check push enabled
        if not device.attributes.push_enabled:
            device_issues.append("Push notifications disabled")
        
        # Check token length (basic validation)
        if len(device.device_info.token) < 20:
            device_issues.append("Suspicious token length")
        
        # Check for missing attributes
        if not device.attributes.timezone:
            device_issues.append("Missing timezone information")
        
        if not device.attributes.locale:
            device_issues.append("Missing locale information")
        
        # Check registration age
        age_days = (datetime.now(timezone.utc) - device.registered_at).days
        if age_days > 90:  # Devices older than 90 days might need token refresh
            device_issues.append(f"Device registration is {age_days} days old")
        
        if device_issues:
            health_report["issues_found"].append({
                "user_id": device.user_id,
                "device_type": device.device_info.type,
                "device_model": device.device_info.device_model,
                "issues": device_issues
            })
        else:
            health_report["healthy_devices"] += 1
    
    # Generate recommendations
    if health_report["issues_found"]:
        health_report["recommendations"].extend([
            "Review devices with disabled push notifications",
            "Update tokens for devices with suspicious token lengths",
            "Collect missing timezone and locale information",
            "Implement periodic token refresh for old registrations"
        ])
    
    health_report["health_score"] = (
        health_report["healthy_devices"] / health_report["total_devices"]
        if health_report["total_devices"] > 0 else 1.0
    )
    
    return health_report

# Monitor device health
health_report = monitor_device_health(all_device_registrations)

print("=== Device Health Monitoring ===")
print(f"Health Score: {health_report['health_score']:.2%}")
print(f"Healthy Devices: {health_report['healthy_devices']}/{health_report['total_devices']}")

if health_report["issues_found"]:
    print(f"\nIssues Found ({len(health_report['issues_found'])}):"))
    for issue in health_report["issues_found"]:
        print(f"  - {issue['user_id']} ({issue['device_type']}): {', '.join(issue['issues'])}")

if health_report["recommendations"]:
    print(f"\nRecommendations:")
    for rec in health_report["recommendations"]:
        print(f"  - {rec}")

## Clean Up and Summary

In [ ]:
# Final summary
print("=== Device Management Summary ===")

print("\n=== Devices Registered ===")
print("SUCCESS: iOS device with APNS integration")
print("SUCCESS: Android device with FCM integration")
print("SUCCESS: Web browser with Web Push API")
print("SUCCESS: Desktop application notifications")
print("SUCCESS: Device data imported from Spark")

print("\n=== Device Operations ===")
print("SUCCESS: Device status lifecycle management")
print("SUCCESS: Push token refresh handling")
print("SUCCESS: Device removal and cleanup")
print("SUCCESS: Cross-platform user tracking")
print("SUCCESS: Batch device operations")

print("\n=== Key Capabilities Demonstrated ===")
print("SUCCESS: Type-safe device registration with validation")
print("SUCCESS: Multi-platform push notification support")
print("SUCCESS: Device lifecycle and status management")
print("SUCCESS: Cross-platform user tracking and analytics")
print("SUCCESS: Batch operations with optimization")
print("SUCCESS: Device health monitoring and diagnostics")
print("SUCCESS: Data integration from Spark DataFrames")
print("SUCCESS: Comprehensive device analytics and reporting")

In [ ]:
# Close the API client connection
client.close()
print("SUCCESS: API client connection closed")

print("\nCOMPLETED: Device management notebook finished successfully!")
print("Ready for advanced tracking operations in the next notebook.")